In [10]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import aiohttp
from pathlib import Path
import time
import pickle
import json

In [2]:
tcp_connector = aiohttp.TCPConnector(limit=None, limit_per_host=0, ssl=False)
session = aiohttp.ClientSession(connector=tcp_connector)

In [3]:
home = str(Path.home())
chrome_download_directory = home + '/chromedriver-download-folder'
chromeOptions = webdriver.ChromeOptions()
chrome_download_directory = ""
prefs = {"download.default_directory" : chrome_download_directory}
chromeOptions.add_experimental_option("prefs",prefs)
chromeOptions.add_argument('ignore-certificate-errors')
driver = webdriver.Chrome(executable_path="/Users/wangrz1/Work/Business/JHUAPL/repos/covid-data-pipeline-local-files/chromedriver", options=chromeOptions)

In [60]:
email = 'roman.transactions@gmail.com'
password = 'fucjyg-wymqak-7mAjko'

In [4]:
url = "https://stockx.com/"
url_selling = "https://stockx.com/selling"

In [5]:
driver.get(url)

In [55]:
def find_and_click(xpath):
    element = driver.find_element_by_xpath(xpath)
    element.click()

In [56]:
find_and_click('//*[@id="nav-login"]')

In [57]:
def find_and_type(xpath, text):
    element = driver.find_element_by_xpath(xpath)
    element.send_keys(text)

In [58]:
find_and_type('//*[@id="email-login"]', email)
find_and_type('//*[@id="password-login"]', password)

In [59]:
find_and_click('//*[@id="login-button-text"]')

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (600, 990)
  (Session info: chrome=88.0.4324.96)


In [ ]:
driver.get(url_selling)

In [ ]:
def scroll_down_until_all_loaded():
    def scroll_down():
        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    SCROLL_PAUSE_TIME = 0.5
    LOADING_PAUSE_TIME = 10
    
    while True:
        last_height = driver.execute_script("return document.body.scrollHeight")
        scroll_down()
        time.sleep(LOADING_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break        

In [70]:
scroll_down_until_all_loaded()

In [79]:
selling_list_table = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[2]/div[2]/div[3]/div[2]/div[2]/div[1]/div/div/div/table')
trs = selling_list_table.find_elements_by_tag_name('tr')
selling_list = []
for index, row in enumerate(trs):
    if index != 0:
        link = row.find_element_by_xpath('.//td[1]/span/div/span[1]/a').get_attribute('href')
        size = row.find_elements_by_xpath('.//td[1]/span/div/span[2]/div')[0].text if row.find_elements_by_xpath('.//td[1]/span/div/span[2]/div') else "N/A"
        ask_price = row.find_element_by_xpath('.//td[2]').text
        highest_bid = row.find_element_by_xpath('.//td[3]').text
        lowest_ask = row.find_element_by_xpath('.//td[4]').text
        spread = row.find_element_by_xpath('.//td[5]').text
        expires = row.find_element_by_xpath('.//td[6]').text
        
        selling_list.append({
            'link': link,
            'size': size,
            'ask_price': ask_price,
            'highest_bid': highest_bid,
            'lowest_ask': lowest_ask,
            'spread': spread,
            'expires': expires
        })

In [ ]:
for item in selling_list:
    item['size'] = item['size'].replace('Size: ', '')
    item['ask_price'] = int(item['ask_price'].replace('$', '')) if item['ask_price'] != '--' else -1
    item['highest_bid'] = int(item['highest_bid'].replace('$', '')) if item['highest_bid'] != '--' else -1
    item['lowest_ask'] = int(item['lowest_ask'].replace('$', '')) if item['lowest_ask'] != '--' else -1
    item['spread'] = int(item['spread'].replace('$', '')) if item['spread'] != '--' else -1
    item['size'] = item['size'].replace('Size: ', '')
    print(item)

In [80]:
print(selling_list)

[{'link': 'https://stockx.com/palace-ye-ham-shirt-indigo?size=S', 'size': 'Size: S', 'ask_price': '$179', 'highest_bid': '$50', 'lowest_ask': '$179', 'spread': ' $128', 'expires': '29 days'}, {'link': 'https://stockx.com/supreme-terry-s-s-zip-up-white?size=S', 'size': 'Size: S', 'ask_price': '$196', 'highest_bid': '$15', 'lowest_ask': '$196', 'spread': ' $181', 'expires': '29 days'}, {'link': 'https://stockx.com/ftp-10-year-jacket-black?size=M', 'size': 'Size: M', 'ask_price': '$149', 'highest_bid': '--', 'lowest_ask': '$149', 'spread': '--', 'expires': '29 days'}, {'link': 'https://stockx.com/supreme-utility-belted-pant-black?size=S', 'size': 'Size: S', 'ask_price': '$153', 'highest_bid': '$61', 'lowest_ask': '$153', 'spread': ' $91', 'expires': '29 days'}, {'link': 'https://stockx.com/palace-bare-storage-jacket-flint-grey?size=M', 'size': 'Size: M', 'ask_price': '$395', 'highest_bid': '$337', 'lowest_ask': '$350', 'spread': ' $58', 'expires': '29 days'}, {'link': 'https://stockx.com/

In [29]:
with open('selling_list.pkl', 'wb') as f:
    pickle.dump(selling_list, f)

In [32]:
with open("selling_list.json", 'w') as f:
    json.dump(selling_list, f)

In [35]:
with open('selling_list.pkl', 'rb') as f:
    selling_list_read = pickle.load(f)

In [31]:
print(selling_list_read)

[{'link': 'https://stockx.com/palace-ye-ham-shirt-indigo?size=S', 'size': 'S', 'ask_price': '$179', 'highest_bid': '$50', 'lowest_ask': '$179', 'spread': ' $128', 'expires': '29 days'}, {'link': 'https://stockx.com/supreme-terry-s-s-zip-up-white?size=S', 'size': 'S', 'ask_price': '$196', 'highest_bid': '$15', 'lowest_ask': '$196', 'spread': ' $181', 'expires': '29 days'}, {'link': 'https://stockx.com/ftp-10-year-jacket-black?size=M', 'size': 'M', 'ask_price': '$149', 'highest_bid': '--', 'lowest_ask': '$149', 'spread': '--', 'expires': '29 days'}, {'link': 'https://stockx.com/supreme-utility-belted-pant-black?size=S', 'size': 'S', 'ask_price': '$153', 'highest_bid': '$61', 'lowest_ask': '$153', 'spread': ' $91', 'expires': '29 days'}, {'link': 'https://stockx.com/palace-bare-storage-jacket-flint-grey?size=M', 'size': 'M', 'ask_price': '$395', 'highest_bid': '$337', 'lowest_ask': '$350', 'spread': ' $58', 'expires': '29 days'}, {'link': 'https://stockx.com/supreme-stone-washed-slim-jean

In [36]:
for item in selling_list_read:
    item['size'] = item['size'].replace('Size: ', '')
    item['ask_price'] = int(item['ask_price'].replace('$', '')) if item['ask_price'] != '--' else -1
    item['highest_bid'] = int(item['highest_bid'].replace('$', '')) if item['highest_bid'] != '--' else -1
    item['lowest_ask'] = int(item['lowest_ask'].replace('$', '')) if item['lowest_ask'] != '--' else -1
    item['spread'] = int(item['spread'].replace('$', '')) if item['spread'] != '--' else -1
    item['size'] = item['size'].replace('Size: ', '')
    print(item)

{'link': 'https://stockx.com/palace-ye-ham-shirt-indigo?size=S', 'size': 'S', 'ask_price': 179, 'highest_bid': 50, 'lowest_ask': 179, 'spread': 128, 'expires': '29 days'}
{'link': 'https://stockx.com/supreme-terry-s-s-zip-up-white?size=S', 'size': 'S', 'ask_price': 196, 'highest_bid': 15, 'lowest_ask': 196, 'spread': 181, 'expires': '29 days'}
{'link': 'https://stockx.com/ftp-10-year-jacket-black?size=M', 'size': 'M', 'ask_price': 149, 'highest_bid': -1, 'lowest_ask': 149, 'spread': -1, 'expires': '29 days'}
{'link': 'https://stockx.com/supreme-utility-belted-pant-black?size=S', 'size': 'S', 'ask_price': 153, 'highest_bid': 61, 'lowest_ask': 153, 'spread': 91, 'expires': '29 days'}
{'link': 'https://stockx.com/palace-bare-storage-jacket-flint-grey?size=M', 'size': 'M', 'ask_price': 395, 'highest_bid': 337, 'lowest_ask': 350, 'spread': 58, 'expires': '29 days'}
{'link': 'https://stockx.com/supreme-stone-washed-slim-jean-fw20-stone-washed-indigo?size=30', 'size': '30', 'ask_price': 229, 

In [37]:
with open("selling_list.json", 'w') as f:
    json.dump(selling_list_read, f)
with open('selling_list.pkl', 'wb') as f:
    pickle.dump(selling_list_read, f)